<br>

# *Imports* e Funções

Inicialmente faz-se necessário importar as bibliotecas que serão necessárias.

In [ ]:
import os
import re
import sys
import shutil
import zipfile
import requests
import geopandas as gpd
from bs4 import BeautifulSoup

<br>

Após isso cria-se as pastas listadas, que armazenarão as informações ao longo desse *script*.

In [ ]:
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))
from paths import *

<br>

Usei a função abaixo para fazer *download* usando o *request*. Ainda, a função pega o nome do arquivo a partir do *Content Disposition*. Peguei a função do *post* [*Downloading Files from URLs in Python*](https://www.codementor.io/@aviaryan/downloading-files-from-urls-in-python-77q3bs0un), aonde tem outros exemplos, com outras finalidades.

In [ ]:
def get_filename_from_cd(cd):
    """
    Get filename from content-disposition
    """
    
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0]

<br>

Após isso a forma de obtenção dos dados:
1. Acessar a página dos metadados do plano de informação e exportar, mantendo armazenadas as informações da origem desse material;
2. Caso seja possível acessar o material cartográfica por *shapefile*, será possível tomar conhecimento disso na página dos metadados;
3. Uma função específica procura o *link* do *shapefile* e faz o download, extração da pasta zipada.
4. Promove-se correções na tabela de atributos e nas projeções geográficas.

<br>

# Função

Uma vez criada uma sequencia de códigos, foi possível definir uma funão que integra todos eles, apresentada abaixo:

In [ ]:
def get_url_shapefile(id_lyr):
    '''
    Pega a URL do shapefile
    
    '''
    
    # Input dos caminhos para os metadados
    url_api = 'http://datageo.ambiente.sp.gov.br/geoportal/catalog/search/resource/details.page?uuid='
    url_meta = '{}{}'.format(url_api, '%7B{}%7D'.format(id_lyr))

    # Abre a página dos metadados
    r = requests.get(url_meta, allow_redirects=True)

    # Parser HTML
    soup = BeautifulSoup(r.content, 'html.parser')
    soup = soup.find_all('a', href=True)

    # Procura Shapefile
    for i in soup:
        text = i.text.split(' ')
        #print(text)
        for j in text:
            #print(j)
            if j in 'Shapefile':                
                url = i['href']
                print('Encontrei o shapefile:\n{}\n'.format(url))
                return url

In [ ]:
id_lyr = '74040682-561A-40B8-BB2F-E188B58088C1'
url = get_url_shapefile(id_lyr)

In [ ]:
def get_metadata_file(id_lyr, output_path, filename=None):
    '''
    Salva arquivo de metadados
    
    '''
    
    # Input dos caminhos para os metadados
    url_api = 'http://datageo.ambiente.sp.gov.br/geoportal/catalog/search/resource/details.page?uuid='
    
    # URL
    url_meta = '{}{}'.format(url_api, '%7B{}%7D'.format(id_lyr))
    print('Página com metadados:\n{}'.format(url_meta))

    # Set Filename
    if filename is None:
        url = get_url_shapefile(id_lyr)
        r = requests.get(url, allow_redirects=True)
        filename_cd = get_filename_from_cd(r.headers.get('content-disposition'))
        filename_metadata = filename_cd.split('.')[0]
    
    else:
        filename_metadata = filename
    
    # Get Metadata File
    r = requests.get(url_meta, allow_redirects=True)
    open(os.path.join(output_path, '{}.html'.format(filename_metadata)), 'wb').write(r.content)
    print('Página HTML dos metadados salva!\n')

In [ ]:
get_metadata_file(id_lyr, output_path, filename='ddd')

In [ ]:
def download_shapefile(id_lyr, output_path, filename=None):
    # Get Shapefile
    url = get_url_shapefile(id_lyr)
    r = requests.get(url, allow_redirects=True)
    
    # Set Filename
    if filename is None:
        # Get Zip filename from CD
        zip_filename = get_filename_from_cd(r.headers.get('content-disposition'))
    
    else:
        zip_filename = f'{filename}.zip'
        
    # Get Zipfile
    open(os.path.join(output_path, zip_filename), 'wb').write(r.content)

    # Unzip
    zipfile_file = os.path.join(output_path, zip_filename)
    zipfile_temp = os.path.join(os.path.dirname(zipfile_file), 'temp')
    os.makedirs(zipfile_temp, exist_ok=True)
    with zipfile.ZipFile(zipfile_file, 'r') as zip_obj:
        zip_obj.extractall(zipfile_temp)

    # Pega o nome do shapefile PRECISA HAVER SOMENTE UM!
    list_shps = [i for i in os.listdir(zipfile_temp) if i.endswith('.shp')]
    print(f'Encontrei {len(list_shps)} arquivos "shapefile", sendo que o primeiro deles é o "{list_shps[0]}"')

    # Read shapefile
    gdf = gpd.read_file(os.path.join(zipfile_temp, list_shps[0]))
    print(gdf.head())

    # Reprojeta
    print(gdf.crs)
    gdf = gdf.to_crs(epsg=4326)
    print(gdf.crs)
    gdf.plot()

    # Excluí pasta temporária
    shutil.rmtree(zipfile_temp)
    return gdf

In [ ]:
download_shapefile('74040682-561A-40B8-BB2F-E188B58088C1', bruto_path)

<br>

# Encerramento

In [ ]:
try:
    if __name__ == '__main__':
        pass
except:
    pass

In [ ]:
import os
from traitlets.config import Config
from nbconvert import PythonExporter
from nbconvert.preprocessors import TagRemovePreprocessor

In [ ]:
# Input
in_filename = '02_get_data_datageo.ipynb'
in_filepath = os.path.abspath(os.path.join(os.getcwd(), '..', 'test', in_filename))
out_filepath = os.path.abspath(os.path.join(src_path, 'get_data_datageo.py'))

print(in_filepath)
print(out_filepath)

In [ ]:
# Import the exporter
c = Config()
c.TagRemovePreprocessor.enabled=True
c.ClearOutputPreprocessor.enabled=True
c.TemplateExporter.exclude_markdown=True
c.TemplateExporter.exclude_code_cell=False
c.TemplateExporter.exclude_input_prompt=True
c.TemplateExporter.exclude_output=True
c.TemplateExporter.exclude_raw=True
c.TagRemovePreprocessor.remove_cell_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_cell',)
c.preprocessors = ['TagRemovePreprocessor']
c.PythonExporter.preprocessors = ['nbconvert.preprocessors.TagRemovePreprocessor']

# Configure and run out exporter
py_exporter = PythonExporter(config=c)
py_exporter.register_preprocessor(TagRemovePreprocessor(config=c), True)

# Configure and run out exporter - returns a tuple - first element with html, second with notebook metadata
body, metadata = PythonExporter(config=c).from_filename(in_filepath)

# Write to output html file
with open(out_filepath,  'w', encoding='utf-8') as f:
    f.write(body)